# Follow virtual linac guide

Follow guide on http://radiotherapyresearchtools.com/doc/api_tutorial.html

* Virtual linac helper files in /Users/robincole/virtualinac specifically file vl_api_helper.py

* after installing boto3 need to set defaults, see https://boto3.readthedocs.io/en/latest/guide/quickstart.html#installation

* Create profile https://console.aws.amazon.com/iam/home#users/robmarkcole

* Config from terminal using 
    
    aws configure 

aws_access_key_id = AKI************

aws_secret_access_key = ZOA*******

region = us-east-1

In [1]:
from __future__ import print_function

In [2]:
import boto3

In [3]:
ec2=boto3.resource('ec2')

First we can get the instances, then iterate over the data structure to find information about each instance:

In [4]:
instances = ec2.instances.filter(Filters=
    [{'Name': 'instance-state-name', 'Values': ['running'] }])

In [5]:
instances 

ec2.instancesCollection(ec2.ServiceResource(), ec2.Instance)

Was an error at next step, had to grant administrator rights to my profile on https://console.aws.amazon.com/iam/home#users/robmarkcole

    ClientError: An error occurred (UnauthorizedOperation) when calling the DescribeInstances operation: You are not authorized to perform this operation.


In [6]:
for instance in instances:
        print(instance.instance_id, instance.public_dns_name)

i-0bba08fc5d69d2b51 ec2-52-23-166-83.compute-1.amazonaws.com


In [7]:
dns_names = []

In [40]:
for instance in instances:
        print(instance.public_dns_name)
        dns_names.append(instance.public_dns_name)
url = 'http://' + dns_names[0]

ec2-52-23-166-83.compute-1.amazonaws.com


## VirtuaLinac API

Now onto http://radiotherapyresearchtools.com/doc/api_tutorial.html#virtualinac-api

We will connect to the VirtuaLinac Amazon Machine Image (AMI) which is in the N. Virginia region.

Had to edit security group to accept incoming HTTP.

<img src="security.png", width=800>

In [9]:
import requests

In [11]:
r = requests.get(url + '/usage') 

In [33]:
for line in r.text.split('\n'):
        if 'Number of jobs' in line:
            print(line)

<p><b>Number of jobs:</b> 0 </p>


Submit calculation

params = {
    'filename'     : 'api_test_1',
    'code_version' : 0,
    'physics_list' : 0,
    'beam_type'    : 0,
    'range_cut' : 10,
    'energy'       : 6.84,
    'energy_spread' : 0.6,
    'spot_size_x'  : 0.7,
    'spot_size_y'  : 0.7,
    'beam_divergence_x' : 0.0,
    'beam_divergence_y' : 0.0,
    'jaw_position_y1' : -20,
    'jaw_position_y2' :  20,
    'jaw_position_x1' : -20,
    'jaw_position_x2' :  20,
    'phantom_bool' : 'True',
    'phantom_size_x' : 500,
    'phantom_size_y' : 500,
    'phantom_size_z' : 400,
    'phantom_voxels_x' : 125,
    'phantom_voxels_y' : 125,
    'phantom_voxels_z' : 100,
    'phantom_position_x' : 0,
    'phantom_position_y' : 0,
    'phantom_position_z' : -5,
    'incident_particles' : 10000,
    'brem_splitting' : None,
    'splitting_factor' : None,
    'phsp_record'  : None
}

In [15]:
r = requests.post(url, params=params)

Calculation runnning

<img src="calc.png", width=800>

In [45]:
resp = requests.get(url + '/phspfiles') # get page with list of output files

In [53]:
Html_file= open("/Users/robincole/Desktop/output.html","w")  # save output to html file and view
Html_file.write(resp.text)
Html_file.close()

<img src="output.png", width=700>

Generate a plot of the data

In [56]:
filename = "testing"

In [57]:
doseparams = {}
doseparams['direction'] = 'z'
doseparams['average'] = 5
doseparams['colormap'] = 'True'
extension = '2d.z.png'
plot_url = url + '/fig/' + filename + '.dose'

In [58]:
extension = '.z_compare.png'

In [59]:
r = requests.get(plot_url, params=doseparams, stream=True)

In [60]:
with open(filename + extension, 'wb') as f:
        for chunk in r.iter_content():
             f.write(chunk)

Plot calculation output

<img src="testing.z_compare.png", width=600>

In [61]:
instances.terminate()  # to terminate

[{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8',
    'date': 'Fri, 09 Sep 2016 14:25:42 GMT',
    'server': 'AmazonEC2',
    'transfer-encoding': 'chunked',
    'vary': 'Accept-Encoding'},
   'HTTPStatusCode': 200,
   'RequestId': '51beea09-e372-4b43-9293-f11f41cc6630',
   'RetryAttempts': 0},
  u'TerminatingInstances': [{u'CurrentState': {u'Code': 32,
     u'Name': 'shutting-down'},
    u'InstanceId': 'i-0bba08fc5d69d2b51',
    u'PreviousState': {u'Code': 16, u'Name': 'running'}}]}]